In [1]:
import sox
import wave
from vosk import Model, KaldiRecognizer, SetLogLevel


def vosk_decode(wave_path):
    SetLogLevel(0)

    wf = wave.open(wave_path, "rb")

    if wf.getnchannels() != 1 or wf.getsampwidth() != 2 or wf.getcomptype() != "NONE":
        print("Audio file must be WAV format mono PCM.")
        exit(1)

    model = Model("vosk-model-small")
    rec = KaldiRecognizer(model, wf.getframerate())
    rec.SetWords(True)

    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if rec.AcceptWaveform(data):
            # print(rec.Result())
            pass
        else:
            print(rec.PartialResult())

    return rec.FinalResult()


In [2]:
wav_path = 'waves/war_1.wav'
wav_info = sox.file_info.info(wav_path)
info = {}
info['Длительность аудио'] = str(round(wav_info['duration'], 2)) + ' с'
info['Число каналов'] = wav_info['channels']
info['Частота дискретизации'] = str(int(wav_info['sample_rate'])) + ' Гц'
print(info)

text = vosk_decode(wav_path)
print(text)

{'Длительность аудио': '6.98 с', 'Число каналов': 1, 'Частота дискретизации': '48000 Гц'}
{
  "partial" : ""
}
{
  "partial" : ""
}
{
  "partial" : ""
}
{
  "partial" : ""
}
{
  "partial" : ""
}
{
  "partial" : ""
}
{
  "partial" : ""
}
{
  "partial" : "война"
}
{
  "partial" : "война"
}
{
  "partial" : "война"
}
{
  "partial" : "война"
}
{
  "partial" : "война"
}
{
  "partial" : "война"
}
{
  "partial" : "война это"
}
{
  "partial" : "война это"
}
{
  "partial" : "война это"
}
{
  "partial" : "война это"
}
{
  "partial" : "война это"
}
{
  "partial" : "война это"
}
{
  "partial" : "война это конфликт между"
}
{
  "partial" : "война это конфликт между"
}
{
  "partial" : "война это конфликт между"
}
{
  "partial" : "война это конфликт между"
}
{
  "partial" : "война это конфликт между"
}
{
  "partial" : "война это конфликт между"
}
{
  "partial" : "война это конфликт между"
}
{
  "partial" : "война это конфликт между"
}
{
  "partial" : "война это конфликт между"
}
{
  "partial" : "война

In [ ]:
import json

json_object = json.loads(text)

my_answer = json_object['text']
right_answer = """конфликт между политическими образованиями — государствами, 
племенами, политическими группировками и так далее, — происходящий на почве различных претензий, 
в форме вооружённого противоборства, военных (боевых) действий между их вооружёнными силами"""
texts = [my_answer, right_answer]

In [7]:
from deeppavlov.core.common.file import read_json
from deeppavlov import build_model, configs

bert_config = read_json(configs.embedder.bert_embedder)
bert_config['metadata']['variables']['BERT_PATH'] = 'rubert'

rubert_model = build_model(bert_config)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package perluniprops to /root/nltk_data...
[nltk_data]   Unzipping misc/perluniprops.zip.
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping corpora/nonbreaking_prefixes.zip.
Some weights of the model checkpoint at /asr/project/rubert were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another ar

In [15]:
# Вычисляем эмбеддинги для каждого ответа
tokens, token_embs, subtokens, subtoken_embs, sent_max_embs, sent_mean_embs, bert_pooler_outputs = rubert_model(texts)

# Эмбеддинги имеют размер: (количества токенов в ответе, 768)
# 768 - длина вектора внутреннего состояния модели RuBERT

sentense_embed = []
for te in token_embs:
    print(te.shape)
    # Усредняем по токенам - словам в предложении
    sentense_embed.append(te.mean(axis=0))


cs = spatial.distance.cosine(sentense_embed[0], sentense_embed[1])
result = int(round(cs*10))

print(f'Оценка - {result} / 10')
      